# $\color{black}{\text{Kapittel 1 - Følger og rekker}}$

# $\color{red}{\text{Kapittel 2 - Integralregning}}$

In [1]:
# -*- coding: utf-8 -*-
import sympy as sp
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, Math, Markdown

# --- SYMBOLSK OPPSETT ---
# Definer symboler. 'oo' er SymPys uendelighet.
x, y = sp.symbols('x y')
oo = sp.oo

# --- HJELPEFUNKSJONER ---

def parse_expression(expr_str):
    """Trygt parser en streng til et SymPy-uttrykk."""
    try:
        # Erstatt standard potensoperator med Pythons versjon
        safe_expr_str = expr_str.replace('^', '**')
        # Sympify konverterer strengen til et matematisk uttrykk
        return sp.sympify(safe_expr_str)
    except (sp.SympifyError, SyntaxError) as e:
        raise ValueError(f"Ugyldig funksjonsuttrykk: '{expr_str}'. Feil: {e}")

def create_numpy_function(expr):
    """Konverterer et SymPy-uttrykk til en rask NumPy-funksjon for plotting."""
    # lambdify er den anbefalte måten å gjøre dette på. Mye tryggere enn eval().
    return sp.lambdify(x, expr, modules=['numpy'])

# --- KJERNEFUNKSJONER (LOGIKK FOR HVER OPERASJON) ---

def handle_derivation(f_expr):
    derivative = sp.diff(f_expr, x)
    display(Math(f"f'(x) = \\frac{{d}}{{dx}} \\left( {sp.latex(f_expr)} \\right) = {sp.latex(derivative)}"))

def handle_indefinite_integral(f_expr):
    antiderivative = sp.integrate(f_expr, x)
    # Legg til "+ C" for å være fullstendig
    display(Math(f"\\int \\left( {sp.latex(f_expr)} \\right) \\,dx = {sp.latex(antiderivative)} + C"))

def handle_definite_integral(f_expr, a, b):
    antiderivative = sp.integrate(f_expr, x)
    display(Math(f"\\text{{1. Finn den antideriverte: }} F(x) = \\int \\left( {sp.latex(f_expr)} \\right) \\,dx = {sp.latex(antiderivative)}"))
    
    result = sp.integrate(f_expr, (x, a, b))
    # Vis utregningen med innsatte grenser
    eval_b = antiderivative.subs(x, b)
    eval_a = antiderivative.subs(x, a)
    display(Math(f"\\text{{2. Evaluer grensene: }} F(b) - F(a) = \\left( {sp.latex(eval_b)} \\right) - \\left( {sp.latex(eval_a)} \\right)"))

    # Vis det endelige svaret, både eksakt og som desimaltall
    display(Math(f"\\text{{Resultat: }} \\int_{{{a}}}^{{{b}}} \\left( {sp.latex(f_expr)} \\right) \\,dx = {sp.latex(result)} \\approx {result.evalf(6)}"))
    plot_functions('Bestemt integral', f_expr=f_expr, a=float(a), b=float(b))

def handle_area_between_graphs(f_expr, g_expr, a, b, find_intersections):
    if find_intersections:
        try:
            # Løs f(x) = g(x) for x, kun reelle løsninger
            intersections = sp.solve(f_expr - g_expr, x, domain=sp.S.Reals)
            intersections.sort()
            if len(intersections) >= 2:
                a, b = intersections[0], intersections[-1]
                a_input.value = float(a)
                b_input.value = float(b)
                display(Markdown(f"**Skjæringspunkter funnet og brukt som grenser:** `a = {a}` og `b = {b}`"))
            else:
                display(Markdown("**Fant færre enn 2 reelle skjæringspunkter. Bruker manuelle grenser.**"))
        except NotImplementedError:
             display(Markdown("**Kunne ikke løse for skjæringspunkter analytisk. Bruk manuelle grenser.**"))

    # Integrer absoluttverdien av differansen for å sikre positivt areal
    integrand = sp.Abs(f_expr - g_expr)
    area = sp.integrate(integrand, (x, a, b))
    
    display(Math(f"\\text{{Areal }} A = \\int_{{{a}}}^{{{b}}} |f(x) - g(x)| \\,dx = \\int_{{{a}}}^{{{b}}} \\left| {sp.latex(f_expr)} - \\left( {sp.latex(g_expr)} \\right) \\right| \\,dx"))
    display(Math(f"A = {sp.latex(area)} \\approx {area.evalf(6)}"))
    plot_functions('Areal mellom grafer', f_expr=f_expr, g_expr=g_expr, a=float(a), b=float(b))

def handle_volume_revolution(f_expr, g_expr, a, b, axis='x'):
    if axis == 'x':
        # Volum ved rotasjon om x-aksen (skive-/vaskemetoden)
        integrand = sp.pi * (f_expr**2 - g_expr**2)
        display(Markdown("### Volum av omdreiningslegeme om x-aksen (Skivemetoden)"))
        display(Math(f"V = \\pi \\int_{{{a}}}^{{{b}}} \\left( (f(x))^2 - (g(x))^2 \\right) \\,dx"))
        display(Math(f"V = \\pi \\int_{{{a}}}^{{{b}}} \\left( \\left({sp.latex(f_expr)}\\right)^2 - \\left({sp.latex(g_expr)}\\right)^2 \\right) \\,dx"))
    else: # axis == 'y'
        # Volum ved rotasjon om y-aksen (sylinderskallmetoden)
        integrand = 2 * sp.pi * x * (f_expr - g_expr)
        display(Markdown("### Volum av omdreiningslegeme om y-aksen (Sylinderskallmetoden)"))
        display(Math(f"V = 2\\pi \\int_{{{a}}}^{{{b}}} x \\left( f(x) - g(x) \\right) \\,dx"))
        display(Math(f"V = 2\\pi \\int_{{{a}}}^{{{b}}} x \\left( \\left({sp.latex(f_expr)}\\right) - \\left({sp.latex(g_expr)}\\right) \\right) \\,dx"))

    volume = sp.integrate(integrand, (x, a, b))
    display(Math(f"V = {sp.latex(volume)} \\approx {volume.evalf(6)}"))
    plot_functions('Volum', f_expr=f_expr, g_expr=g_expr, a=float(a), b=float(b), title_extra=f"om {axis}-aksen")

def handle_numerical_integration(f_str, a, b):
    # Bruk SciPy.quad for rask og nøyaktig numerisk integrasjon
    f_numpy = lambda x_val: eval(f_str, {"x": x_val, "np": np})
    result, error = integrate.quad(f_numpy, a, b)
    display(Markdown(f"**Resultat:** `{result}`"))
    display(Markdown(f"**Estimert feilmargin:** `{error}`"))
    
    # For plotting må vi parse uttrykket til SymPy først
    f_expr = parse_expression(f_str)
    plot_functions('Numerisk integral', f_expr=f_expr, a=a, b=b)

# --- PLOTTEFUNKSJON ---

def plot_functions(op_name, f_expr, g_expr=None, a=0, b=1, title_extra=""):
    # Lag numeriske funksjoner for plotting
    f_numpy = create_numpy_function(f_expr)
    
    # Lag et tett rutenett av x-verdier for en jevn graf
    x_vals = np.linspace(a, b, 500)
    y_vals_f = f_numpy(x_vals)

    plt.figure(figsize=(10, 6))
    plt.plot(x_vals, y_vals_f, label=f'f(x) = {f_expr}', color='blue')
    
    plot_title = f"Visualisering av {op_name} {title_extra}"
    
    if g_expr is not None:
        g_numpy = create_numpy_function(g_expr)
        y_vals_g = g_numpy(x_vals)
        plt.plot(x_vals, y_vals_g, label=f'g(x) = {g_expr}', color='green')
        
        # Fyll området mellom grafene
        plt.fill_between(x_vals, y_vals_f, y_vals_g, where=(y_vals_f > y_vals_g), 
                         interpolate=True, color='purple', alpha=0.3, label='Areal (f > g)')
        plt.fill_between(x_vals, y_vals_f, y_vals_g, where=(y_vals_g >= y_vals_f), 
                         interpolate=True, color='orange', alpha=0.3, label='Areal (g >= f)')

        # Finn og plott skjæringspunkter i intervallet
        try:
            intersections = sp.solve(f_expr - g_expr, x, domain=sp.S.Reals)
            real_intersections_in_range = [i for i in intersections if a <= i <= b]
            if real_intersections_in_range:
                y_intersections = f_numpy(np.array(real_intersections_in_range, dtype=float))
                plt.scatter(real_intersections_in_range, y_intersections, color='red', zorder=5, label='Skjæringspunkter')
        except NotImplementedError:
            pass # Ignorer om SymPy ikke kan løse ligningen

    elif op_name in ['Bestemt integral', 'Numerisk integral']:
        plt.fill_between(x_vals, y_vals_f, alpha=0.3, color='orange', label='Integrasjonsområde')

    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.title(plot_title)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.grid(True)
    plt.ylim(min(y_vals_f.min(), 0) - 1, y_vals_f.max() + 1) # Juster y-aksen
    plt.xlim(a - 0.1*(b-a), b + 0.1*(b-a)) # Litt pusterom på x-aksen
    plt.show()

# --- OPPSETT AV WIDGETS (BRUKERGRENSESNITT) ---

# Beskrivelse av verktøyet
display(Markdown("# R2 Kalkulus-verktøy"))
display(Markdown("Dette verktøyet utfører symbolske og numeriske beregninger for derivasjon og integrasjon. "
                 "Bruk standard Python-syntaks for funksjoner, f.eks. `x**2` for $x^2$, `sp.sin(x)` for $\\sin(x)$, `sp.exp(x)` for $e^x$. "
                 "For uendelig, skriv `oo` (to små o-er)."))

# Hovedmeny
operation = widgets.Dropdown(
    options=[
        'Derivasjon',
        'Ubestemt integral',
        'Bestemt integral',
        'Areal mellom grafer',
        'Volum om x-aksen',
        'Volum om y-aksen',
        'Numerisk integral'
    ],
    description='Velg operasjon:',
    style={'description_width': 'initial'}
)

# Inndatafelt
function_input = widgets.Text(value='sin(x)', description='f(x):', layout={'width': '400px'})
function_g_input = widgets.Text(value='cos(x)', description='g(x):', layout={'width': '400px'})
a_input = widgets.Text(value='0', description='a (nedre grense):')
b_input = widgets.Text(value='pi/2', description='b (øvre grense):')
find_intersections_checkbox = widgets.Checkbox(value=False, description='Finn grenser automatisk')
run_button = widgets.Button(description="Kjør beregning", button_style='success')

# Område for output
output_area = widgets.Output()

# Ordne widgets i bokser for bedre layout
input_widgets = widgets.VBox([
    operation,
    function_input,
    function_g_input,
    widgets.HBox([a_input, b_input]),
    find_intersections_checkbox
])

# --- LOGIKK FOR BRUKERGRENSESNITT ---

def update_ui(*args):
    """Viser/skjuler widgets basert på valgt operasjon."""
    op = operation.value
    function_g_input.layout.display = 'none'
    a_input.layout.display = 'none'
    b_input.layout.display = 'none'
    find_intersections_checkbox.layout.display = 'none'

    if op == 'Areal mellom grafer':
        function_g_input.layout.display = 'flex'
        a_input.layout.display = 'flex'
        b_input.layout.display = 'flex'
        find_intersections_checkbox.layout.display = 'flex'
    elif op in ['Bestemt integral', 'Numerisk integral']:
        a_input.layout.display = 'flex'
        b_input.layout.display = 'flex'
    elif op in ['Volum om x-aksen', 'Volum om y-aksen']:
        function_g_input.description = "g(x) (nedre kurve):"
        function_g_input.value = "0" # Standard er rotasjon av areal under én graf
        function_g_input.layout.display = 'flex'
        a_input.layout.display = 'flex'
        b_input.layout.display = 'flex'

operation.observe(update_ui, names='value')

def run_calculation_on_click(b):
    """Hovedfunksjon som kalles når knappen trykkes."""
    with output_area:
        clear_output(wait=True)
        try:
            op = operation.value
            f_str = function_input.value
            f_expr = parse_expression(f_str)

            # Hent grenser kun om de trengs
            if op not in ['Derivasjon', 'Ubestemt integral']:
                a = parse_expression(a_input.value)
                b = parse_expression(b_input.value)
            
            # Hent g(x) kun om det trengs
            if op in ['Areal mellom grafer', 'Volum om x-aksen', 'Volum om y-aksen']:
                g_expr = parse_expression(function_g_input.value)

            # Kall riktig logikkfunksjon basert på valg
            if op == 'Derivasjon':
                handle_derivation(f_expr)
            elif op == 'Ubestemt integral':
                handle_indefinite_integral(f_expr)
            elif op == 'Bestemt integral':
                handle_definite_integral(f_expr, a, b)
            elif op == 'Areal mellom grafer':
                handle_area_between_graphs(f_expr, g_expr, a, b, find_intersections_checkbox.value)
            elif op == 'Volum om x-aksen':
                handle_volume_revolution(f_expr, g_expr, a, b, axis='x')
            elif op == 'Volum om y-aksen':
                handle_volume_revolution(f_expr, g_expr, a, b, axis='y')
            elif op == 'Numerisk integral':
                # Numerisk trenger den originale strengen
                handle_numerical_integration(f_str, float(a), float(b))

        except Exception as e:
            # Vis feilmelding på en pen måte
            print(f"En feil oppstod: {e}")

run_button.on_click(run_calculation_on_click)

# Vis grensesnittet
display(input_widgets)
display(run_button)
display(output_area)

# Initialiser UI-visningen
update_ui()

# R2 Kalkulus-verktøy

Dette verktøyet utfører symbolske og numeriske beregninger for derivasjon og integrasjon. Bruk standard Python-syntaks for funksjoner, f.eks. `x**2` for $x^2$, `sp.sin(x)` for $\sin(x)$, `sp.exp(x)` for $e^x$. For uendelig, skriv `oo` (to små o-er).

Button(button_style='success', description='Kjør beregning', style=ButtonStyle())

Output()

In [2]:
# -*- coding: utf-8 -*-
import sympy as sp
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, Math, Markdown

# --- SYMBOLSK OPPSETT ---
# Definer symboler. 'oo' er SymPys uendelighet.
x, y = sp.symbols('x y')
oo = sp.oo

# --- HJELPEFUNKSJONER ---

def parse_expression(expr_str):
    """Trygt parser en streng til et SymPy-uttrykk."""
    try:
        # Erstatt standard potensoperator med Pythons versjon
        safe_expr_str = expr_str.replace('^', '**')
        # Sympify konverterer strengen til et matematisk uttrykk
        return sp.sympify(safe_expr_str)
    except (sp.SympifyError, SyntaxError) as e:
        raise ValueError(f"Ugyldig funksjonsuttrykk: '{expr_str}'. Feil: {e}")

def create_numpy_function(expr):
    """Konverterer et SymPy-uttrykk til en rask NumPy-funksjon for plotting."""
    # lambdify er den anbefalte måten å gjøre dette på. Mye tryggere enn eval().
    return sp.lambdify(x, expr, modules=['numpy'])

# --- KJERNEFUNKSJONER (LOGIKK FOR HVER OPERASJON) ---

def handle_derivation(f_expr):
    derivative = sp.diff(f_expr, x)
    display(Math(f"f'(x) = \\frac{{d}}{{dx}} \\left( {sp.latex(f_expr)} \\right) = {sp.latex(derivative)}"))

def handle_indefinite_integral(f_expr):
    antiderivative = sp.integrate(f_expr, x)
    # Legg til "+ C" for å være fullstendig
    display(Math(f"\\int \\left( {sp.latex(f_expr)} \\right) \\,dx = {sp.latex(antiderivative)} + C"))

def handle_definite_integral(f_expr, a, b):
    antiderivative = sp.integrate(f_expr, x)
    display(Math(f"\\text{{1. Finn den antideriverte: }} F(x) = \\int \\left( {sp.latex(f_expr)} \\right) \\,dx = {sp.latex(antiderivative)}"))
    
    result = sp.integrate(f_expr, (x, a, b))
    # Vis utregningen med innsatte grenser
    eval_b = antiderivative.subs(x, b)
    eval_a = antiderivative.subs(x, a)
    display(Math(f"\\text{{2. Evaluer grensene: }} F(b) - F(a) = \\left( {sp.latex(eval_b)} \\right) - \\left( {sp.latex(eval_a)} \\right)"))

    # Vis det endelige svaret, både eksakt og som desimaltall
    display(Math(f"\\text{{Resultat: }} \\int_{{{a}}}^{{{b}}} \\left( {sp.latex(f_expr)} \\right) \\,dx = {sp.latex(result)} \\approx {result.evalf(6)}"))
    plot_functions('Bestemt integral', f_expr=f_expr, a=float(a), b=float(b))

def handle_area_between_graphs(f_expr, g_expr, a, b, find_intersections):
    if find_intersections:
        try:
            # Løs f(x) = g(x) for x, kun reelle løsninger
            intersections = sp.solve(f_expr - g_expr, x, domain=sp.S.Reals)
            intersections.sort()
            if len(intersections) >= 2:
                a, b = intersections[0], intersections[-1]
                a_input.value = str(a) # Oppdater UI
                b_input.value = str(b) # Oppdater UI
                display(Markdown(f"**Skjæringspunkter funnet og brukt som grenser:** `a = {a}` og `b = {b}`"))
            else:
                display(Markdown("**Fant færre enn 2 reelle skjæringspunkter. Bruker manuelle grenser.**"))
        except NotImplementedError:
             display(Markdown("**Kunne ikke løse for skjæringspunkter analytisk. Bruk manuelle grenser.**"))

    # Integrer absoluttverdien av differansen for å sikre positivt areal
    integrand = sp.Abs(f_expr - g_expr)
    area = sp.integrate(integrand, (x, a, b))
    
    display(Math(f"\\text{{Areal }} A = \\int_{{{a}}}^{{{b}}} |f(x) - g(x)| \\,dx = \\int_{{{a}}}^{{{b}}} \\left| {sp.latex(f_expr)} - \\left( {sp.latex(g_expr)} \\right) \\right| \\,dx"))
    display(Math(f"A = {sp.latex(area)} \\approx {area.evalf(6)}"))
    plot_functions('Areal mellom grafer', f_expr=f_expr, g_expr=g_expr, a=float(a), b=float(b))

def handle_volume_revolution(f_expr, g_expr, a, b, axis='x'):
    if axis == 'x':
        # Volum ved rotasjon om x-aksen (skive-/vaskemetoden)
        integrand = sp.pi * (f_expr**2 - g_expr**2)
        display(Markdown("### Volum av omdreiningslegeme om x-aksen (Skivemetoden)"))
        display(Math(f"V = \\pi \\int_{{{a}}}^{{{b}}} \\left( (f(x))^2 - (g(x))^2 \\right) \\,dx"))
        display(Math(f"V = \\pi \\int_{{{a}}}^{{{b}}} \\left( \\left({sp.latex(f_expr)}\\right)^2 - \\left({sp.latex(g_expr)}\\right)^2 \\right) \\,dx"))
    else: # axis == 'y'
        # Volum ved rotasjon om y-aksen (sylinderskallmetoden)
        integrand = 2 * sp.pi * x * (f_expr - g_expr)
        display(Markdown("### Volum av omdreiningslegeme om y-aksen (Sylinderskallmetoden)"))
        display(Math(f"V = 2\\pi \\int_{{{a}}}^{{{b}}} x \\left( f(x) - g(x) \\right) \\,dx"))
        display(Math(f"V = 2\\pi \\int_{{{a}}}^{{{b}}} x \\left( \\left({sp.latex(f_expr)}\\right) - \\left({sp.latex(g_expr)}\\right) \\right) \\,dx"))

    volume = sp.integrate(integrand, (x, a, b))
    display(Math(f"V = {sp.latex(volume)} \\approx {volume.evalf(6)}"))
    plot_functions('Volum', f_expr=f_expr, g_expr=g_expr, a=float(a), b=float(b), title_extra=f"om {axis}-aksen")

def handle_numerical_integration(f_expr, a, b):
    # OPPDATERT: Bruker lambdify for sikkerhet og konsistens.
    f_numpy = create_numpy_function(f_expr)
    
    # Bruk SciPy.quad for rask og nøyaktig numerisk integrasjon
    result, error = integrate.quad(f_numpy, a, b)
    display(Markdown(f"**Numerisk resultat (via SciPy):** `{result}`"))
    display(Markdown(f"**Estimert feilmargin:** `{error}`"))
    
    plot_functions('Numerisk integral', f_expr=f_expr, a=a, b=b)

# --- PLOTTEFUNKSJON ---

def plot_functions(op_name, f_expr, g_expr=None, a=0, b=1, title_extra=""):
    f_numpy = create_numpy_function(f_expr)
    x_vals = np.linspace(a, b, 500)
    y_vals_f = f_numpy(x_vals)

    plt.figure(figsize=(10, 6))
    plt.plot(x_vals, y_vals_f, label=f'$f(x) = {sp.latex(f_expr)}$', color='blue')
    
    plot_title = f"Visualisering av {op_name} {title_extra}"
    
    if g_expr is not None:
        g_numpy = create_numpy_function(g_expr)
        y_vals_g = g_numpy(x_vals)
        plt.plot(x_vals, y_vals_g, label=f'$g(x) = {sp.latex(g_expr)}$', color='green')
        
        plt.fill_between(x_vals, y_vals_f, y_vals_g, where=(y_vals_f > y_vals_g), 
                         interpolate=True, color='purple', alpha=0.3, label='Areal (f > g)')
        plt.fill_between(x_vals, y_vals_f, y_vals_g, where=(y_vals_g >= y_vals_f), 
                         interpolate=True, color='orange', alpha=0.3, label='Areal (g >= f)')

        try:
            intersections = sp.solve(f_expr - g_expr, x, domain=sp.S.Reals)
            real_intersections_in_range = [i for i in intersections if a <= i <= b]
            if real_intersections_in_range:
                y_intersections = f_numpy(np.array(real_intersections_in_range, dtype=float))
                plt.scatter(real_intersections_in_range, y_intersections, color='red', zorder=5, label='Skjæringspunkter')
        except NotImplementedError:
            pass 

    elif op_name in ['Bestemt integral', 'Numerisk integral']:
        plt.fill_between(x_vals, y_vals_f, alpha=0.3, color='orange', label='Integrasjonsområde')

    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.title(plot_title)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.grid(True)
    plt.ylim(min(y_vals_f.min(), 0) - 1, y_vals_f.max() + 1)
    plt.xlim(a - 0.1*(b-a), b + 0.1*(b-a))
    plt.show()

# --- OPPSETT AV WIDGETS (BRUKERGRENSESNITT) ---

display(Markdown("# R2 Kalkulus-verktøy"))
display(Markdown("Bruk standard Python/SymPy-syntaks, f.eks. `x**2` for $x^2$, `sin(x)`, `exp(x)` for $e^x$, og `pi` for $\\pi$. "
                 "For uendelig, skriv `oo` (to små o-er)."))

operation = widgets.Dropdown(
    options=[
        'Derivasjon', 'Ubestemt integral', 'Bestemt integral', 'Areal mellom grafer',
        'Volum om x-aksen', 'Volum om y-aksen', 'Numerisk integral'
    ],
    description='Velg operasjon:', style={'description_width': 'initial'}
)

function_input = widgets.Text(value='sin(x)', description='f(x):', layout={'width': '400px'})
function_g_input = widgets.Text(value='0', description='g(x):', layout={'width': '400px'})
a_input = widgets.Text(value='0', description='a (nedre grense):')
b_input = widgets.Text(value='pi', description='b (øvre grense):')
find_intersections_checkbox = widgets.Checkbox(value=False, description='Finn grenser automatisk')
run_button = widgets.Button(description="Kjør beregning", button_style='success')
output_area = widgets.Output()

input_widgets = widgets.VBox([
    operation, function_input, function_g_input,
    widgets.HBox([a_input, b_input]), find_intersections_checkbox
])

# --- LOGIKK FOR BRUKERGRENSESNITT ---

def update_ui(*args):
    op = operation.value
    function_g_input.layout.display = 'none'
    a_input.layout.display = 'none'
    b_input.layout.display = 'none'
    find_intersections_checkbox.layout.display = 'none'

    if op == 'Areal mellom grafer':
        function_g_input.description = "g(x):"
        function_g_input.layout.display = 'flex'
        a_input.layout.display = 'flex'
        b_input.layout.display = 'flex'
        find_intersections_checkbox.layout.display = 'flex'
    elif op in ['Bestemt integral', 'Numerisk integral']:
        a_input.layout.display = 'flex'
        b_input.layout.display = 'flex'
    elif op in ['Volum om x-aksen', 'Volum om y-aksen']:
        function_g_input.description = "g(x) (indre kurve):"
        function_g_input.layout.display = 'flex'
        a_input.layout.display = 'flex'
        b_input.layout.display = 'flex'

operation.observe(update_ui, names='value')

def run_calculation_on_click(b):
    with output_area:
        clear_output(wait=True)
        try:
            op = operation.value
            f_expr = parse_expression(function_input.value)

            a, b_val, g_expr = None, None, None
            if op not in ['Derivasjon', 'Ubestemt integral']:
                a = parse_expression(a_input.value)
                b_val = parse_expression(b_input.value)
            if op in ['Areal mellom grafer', 'Volum om x-aksen', 'Volum om y-aksen']:
                g_expr = parse_expression(function_g_input.value)

            if op == 'Derivasjon': handle_derivation(f_expr)
            elif op == 'Ubestemt integral': handle_indefinite_integral(f_expr)
            elif op == 'Bestemt integral': handle_definite_integral(f_expr, a, b_val)
            elif op == 'Areal mellom grafer': handle_area_between_graphs(f_expr, g_expr, a, b_val, find_intersections_checkbox.value)
            elif op == 'Volum om x-aksen': handle_volume_revolution(f_expr, g_expr, a, b_val, axis='x')
            elif op == 'Volum om y-aksen': handle_volume_revolution(f_expr, g_expr, a, b_val, axis='y')
            elif op == 'Numerisk integral':
                # OPPDATERT: Sender SymPy-uttrykket i stedet for strengen
                handle_numerical_integration(f_expr, float(a), float(b_val))

        except Exception as e:
            print(f"En feil oppstod: {e}")

run_button.on_click(run_calculation_on_click)

# --- START PROGRAMMET ---
display(input_widgets, run_button, output_area)
update_ui()

# R2 Kalkulus-verktøy

Bruk standard Python/SymPy-syntaks, f.eks. `x**2` for $x^2$, `sin(x)`, `exp(x)` for $e^x$, og `pi` for $\pi$. For uendelig, skriv `oo` (to små o-er).

Button(button_style='success', description='Kjør beregning', style=ButtonStyle())

Output()

# $\color{green}{\text{Kapittel 3 - Integrasjonsmetoder}}$

# $\color{blue}{\text{Kapittel 4 - Vektorer}}$

# $\color{lightblue}{\text{Kapittel 5 - Trigonometri}}$

# $\color{coral}{\text{Kapittel 6 - Funksjoner og kurver}}$